<a href="https://colab.research.google.com/github/nceder/qpb4e/blob/main/code/Chapter%2019/Chapter_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 19. Packages

# 19.3 Implementing the mathproj package

In [ ]:
! mkdir mathproj
! mkdir mathproj/comp
! mkdir mathproj/comp/numeric
open("mathproj/__init__.py", "w").write("""print("Hello from mathproj init")
__all__ = ['comp']
version = 1.03""")

open("mathproj/comp/__init__.py", "w").write("""__all__ = ['c1']
print("Hello from mathproj.comp init")""")
open("mathproj/comp/numeric/__init__.py", "w").write("""print("Hello from numeric init")""")
open("mathproj/comp/c1.py", "w").write("""x = 1.00""")
open("mathproj/comp/numeric/n1.py", "w").write("""# File mathproj/comp/numeric/n1.py

from mathproj import version
from mathproj.comp import c1
from mathproj.comp.numeric.n2 import h
def g():
    print("version is", version)
    print(h())  """)
open("mathproj/comp/numeric/n2.py", "w").write("""def h():
    return "Called function h in module n2"
""")



mkdir: cannot create directory ‘mathproj’: File exists
mkdir: cannot create directory ‘mathproj/comp’: File exists
mkdir: cannot create directory ‘mathproj/comp/numeric’: File exists


53

In [ ]:
# File mathproj/__init__.py

print("Hello from mathproj init")
__all__ = ['comp']
version = 1.03

Hello from mathproj init


In [ ]:
# File mathproj/comp/__init__.py

__all__ = ['c1']
print("Hello from mathproj.comp init")

Hello from mathproj.comp init


In [ ]:
# File mathproj/comp/c1.py

x = 1.00

In [ ]:
# File mathproj/comp/numeric/__init__.py

print("Hello from numeric init")

Hello from numeric init


In [ ]:
# File mathproj/comp/numeric/n1.py

from mathproj import version
from mathproj.comp import c1
from mathproj.comp.numeric.n2 import h
def g():
    print("version is", version)
    print(h())

Hello from mathproj init
Hello from mathproj.comp init
Hello from numeric init


In [ ]:
# File mathproj/comp/numeric/n2.py

def h():
    return "Called function h in module n2"

## 19.3.2 Basic use of the mathproj package

In [ ]:
import mathproj

In [ ]:
mathproj.version

1.03

## 19.3.3 Loading subpackages and submodules

In [ ]:
mathproj.comp.numeric.n1

AttributeError: module 'mathproj.comp.numeric' has no attribute 'n1'

In [ ]:
import mathproj.comp.numeric.n1

In [ ]:
mathproj.comp.numeric.n1.g()

In [ ]:
mathproj.comp

In [ ]:
mathproj.comp.numeric

# 19.4 The `__all__` attribute

In [ ]:
# use the Runtime menu, and Restart session

from mathproj import *

Hello from mathproj init
Hello from mathproj.comp init


In [ ]:
comp

<module 'mathproj.comp' from '/content/mathproj/comp/__init__.py'>

In [ ]:
c1

NameError: name 'c1' is not defined

In [ ]:
from mathproj.comp import c1
c1

<module 'mathproj.comp.c1' from '/content/mathproj/comp/c1.py'>

# 19.6 Lab 19: Create a package
In chapter 14, you added error handling to the text cleaning and word frequency counting module that you created in chapter 10. Refactor that code into a package containing at least one module for the cleaning functions, and another for the processing functions, and (optionally) one for custom exceptions (if you have any). Then write a simple main function that uses all the modules in the package.

In [1]:
!wget https://raw.githubusercontent.com/nceder/qpb4e/main/code/Chapter%2006/moby_01.txt &> null  && echo Downloaded

Downloaded


## Author's solution

In [2]:
! mkdir word_count

### `__init__.py`

In [3]:
%%writefile word_count/__init__.py
#__init__.py

from word_count.exceptions import EmptyStringError
from word_count.cleaning import clean_line, get_words
from word_count.counter import count_words, word_stats

Writing word_count/__init__.py


### `exceptions.py`

In [4]:
%%writefile word_count/exceptions.py
# exceptions.py

class EmptyStringError(Exception):
    pass

Writing word_count/exceptions.py


### `cleaning.py`

In [6]:
%%writefile word_count/cleaning.py
# cleaning.py

from word_count.exceptions import EmptyStringError

punct = str.maketrans("",  "", "!.,:;-?")

def clean_line(line):
    """changes case and removes punctuation"""

    # raise exception if line is empty
    # uncomment to test EmptyStringError
    #if not line.strip():
    #    raise EmptyStringError()

    # make all one case
    cleaned_line = line.lower()

    # remove punctuation
    cleaned_line = cleaned_line.translate(punct)
    return cleaned_line


def get_words(line):
    """splits line into words, and rejoins with newlines"""
    words = line.split()
    return "\n".join(words) + "\n"

Overwriting word_count/cleaning.py


### `counter.py`

In [7]:
%%writefile word_count/counter.py
# counter.py

def count_words(words):
    """takes list of cleaned words, returns count dictionary"""
    word_count = {}
    for word in words:
        try:
            count = word_count.setdefault(word, 0)
        except TypeError:
            #if 'word' is not hashable, skip to next word.
            pass
        word_count[word] += 1
    return word_count


def word_stats(word_count):
    """Takes word count dictionary, returns top and bottom five entries"""
    word_list = list(word_count.items())
    word_list.sort(key=lambda x: x[1])
    try:
        least_common = word_list[:5]
        most_common = word_list[-1:-6:-1]
    except IndexError as e:
        # if list is empty or too short, just return list
        least_common = word_list
        most_common = list(reversed(word_list))

    return most_common, least_common

Writing word_count/counter.py


### Code to use package

In [8]:
import word_count

if __name__ == "__main__":
    with (open("moby_01.txt") as infile,
            open("moby_01_clean.txt", "w") as outfile):
        for line in infile:
            cleaned_line = word_count.clean_line(line)

            cleaned_words = word_count.get_words(cleaned_line)

            # write all words for line
            outfile.write(cleaned_words)

    moby_words = []
    with open('moby_01_clean.txt') as infile:
        for word in infile:
            if word.strip():
                moby_words.append(word.strip())

    word_counter = word_count.count_words(moby_words)

    most, least = word_count.word_stats(word_counter)
    print("Most common words:")
    for word in most:
        print(word)
    print("\nLeast common words:")
    for word in least:
        print(word)

Most common words:
('the', 14)
('i', 9)
('and', 9)
('of', 8)
('is', 7)

Least common words:
('call', 1)
('ishmael', 1)
('years', 1)
('ago', 1)
('never', 1)


# Copilot solution

### Prompt

Refactor the code in word_processing.py into a package containing at least one module for the cleaning functions, and another for the processing functions, and (optionally) one for custom exceptions (if you have any). The refactor main.py to use the new package.

### word_processing/cleaning.py

In [ ]:
def clean_line(line):
    # Your cleaning function implementation here
    Pass

### word_processing/processing.py

In [ ]:
from .cleaning import clean_line

def write_words_to_file(words, output_file):
    # Your write_words_to_file function implementation here
    pass

def print_common_words(word_counts, n):
    # Your print_common_words function implementation here
    pass

def print_least_common_words(word_counts, n):
    # Your print_least_common_words function implementation here
    pass

def process_file(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file:
        with open(output_file_path, 'w') as output_file:
            for line in input_file:
                cleaned_line = clean_line(line)
                words = cleaned_line.split()
                write_words_to_file(words, output_file)

### word_processing/exceptions.py

In [ ]:
# If you have any custom exceptions, define them here

### main.py

In [ ]:
from word_processing.processing import process_file

# Use the process_file function from the word_processing package
process_file('input.txt', 'output.txt')

### Additional prompt

Also add a `__init__.py` that appropriately imports the package's functions.

### `__init__.py`

In [ ]:
from .cleaning import clean_line
from .processing import (write_words_to_file, print_common_words,
                        print_least_common_words, process_file)

# Colaboratory solution

### Prompt

Refactor the code in module created in cell above into a package containing at least one module for the cleaning functions, and another for the processing functions, and (optionally) one for custom exceptions (if you have any). Then refactor `main()` to use the new package.

### Code from chapter 10 solution to refactor

In [ ]:
# Generated by colaboratory
# prompt: Package the functions in the cell above as a standalone module so that
#         the module can be used by other code. Then write the code to use the
#         module to achieve the same result as the cell above.

# Save the functions as a module named "text_processing.py"
with open("text_processing.py", "w") as f:
    f.write(r"""
def clean_line(line):
    # Convert the line to lowercase
    line = line.lower()

    # Remove punctuation from the line
    line = line.translate(str.maketrans('', '', string.punctuation))

    return line

def write_words_to_file(words, output_file):
    # Write each word to the output file
    for word in words:
        output_file.write(word + '\n')

def count_word_occurrences(words):
    # Count the occurrences of each word using a dictionary
    word_counts = {}
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    return word_counts

def print_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n most common words and their occurrences
    print(f"{n} most common words:")
    for word, count in sorted_word_counts[:n]:
        print(f"{word}: {count}")

def print_least_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n least common words and their occurrences
    print(f"{n} least common words:")
    for word, count in sorted_word_counts[-n:]:
        print(f"{word}: {count}")

def process_file(input_file_path, output_file_path):
    # Open the input file for reading
    with open(input_file_path, 'r') as input_file:
        # Open the output file for writing
        with open(output_file_path, 'w') as output_file:
            # Iterate over each line in the input file
            for line in input_file:
                # Clean the line
                cleaned_line = clean_line(line)

                # Split the line into words
                words = cleaned_line.split()

                # Write the words to the output file
                write_words_to_file(words, output_file)
""")

# Use the module
import text_processing

input_file_path = 'moby_01.txt'
output_file_path = 'moby_01_clean.txt'

# Process the file
text_processing.process_file(input_file_path, output_file_path)

# Open the file
with open(output_file_path, 'r') as file:
    # Read the file content
    content = file.read()

# Split the content into words
words = content.split()

# Count word occurrences
word_counts = text_processing.count_word_occurrences(words)

# Print the five most common words and their occurrences
text_processing.print_common_words(word_counts, 5)

# Print the five least common words and their occurrences
text_processing.print_least_common_words(word_counts, 5)

### Colaboratory non-working solution

In [9]:
 # prompt: Refactor the code in module created in cell above into a package
 #        containing at least one module for the cleaning functions, and another
 #        for the processing functions, and (optionally) one for custom exceptions
 #         (if you have any). Then refactor main() to use the new package.

# Create a directory for the package
!mkdir text_processor

# Create a file for the cleaning functions
%%writefile text_processor/cleaning.py
def clean_line(line):
    # Convert the line to lowercase
    line = line.lower()

    # Remove punctuation from the line
    line = line.translate(str.maketrans('', '', string.punctuation))

    return line

# Create a file for the processing functions
%%writefile text_processor/processing.py
def write_words_to_file(words, output_file):
    # Write each word to the output file
    for word in words:
        output_file.write(word + '\n')

def count_word_occurrences(words):
    # Count the occurrences of each word using a dictionary
    word_counts = {}
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    return word_counts

def print_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n most common words and their occurrences
    print(f"{n} most common words:")
    for word, count in sorted_word_counts[:n]:
        print(f"{word}: {count}")

def print_least_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n least common words and their occurrences
    print(f"{n} least common words:")
    for word, count in sorted_word_counts[-n:]:
        print(f"{word}: {count}")

def process_file(input_file_path, output_file_path):
    # Open the input file for reading
    with open(input_file_path, 'r') as input_file:
        # Open the output file for writing
        with open(output_file_path, 'w') as output_file:
            # Iterate over each line in the input file
            for line in input_file:
                # Clean the line
                cleaned_line = clean_line(line)

                # Split the line into words
                words = cleaned_line.split()

                # Write the words to the output file
                write_words_to_file(words, output_file)

# Create an __init__.py file to make the directory a package
%%writefile text_processor/__init__.py
from .cleaning import clean_line
from .processing import process_file, count_word_occurrences, print_common_words, print_least_common_words

# Refactor main() to use the new package
%%writefile main.py
import text_processor

input_file_path = 'moby_01.txt'
output_file_path = 'moby_01_clean.txt'

# Process the file
text_processor.process_file(input_file_path, output_file_path)

# Open the file
with open(output_file_path, 'r') as file:
    # Read the file content
    content = file.read()

# Split the content into words
words = content.split()

# Count word occurrences
word_counts = text_processor.count_word_occurrences(words)

# Print the five most common words and their occurrences
text_processor.print_common_words(word_counts, 5)

# Print the five least common words and their occurrences
text_processor.print_least_common_words(word_counts, 5)

# Run the main script
!python main.py


UsageError: Line magic function `%%writefile` not found.


## Colaboratory solution broken into cells

In [10]:
# Create a directory for the package
!mkdir text_processor


mkdir: cannot create directory ‘text_processor’: File exists


In [1]:
# Create a file for the cleaning functions
%%writefile text_processor/cleaning.py

#### THIS IMPORT WAS ADDED BY AUTHOR
import string

def clean_line(line):
    # Convert the line to lowercase
    line = line.lower()

    # Remove punctuation from the line
    line = line.translate(str.maketrans('', '', string.punctuation))

    return line

Overwriting text_processor/cleaning.py


In [2]:
# Create a file for the processing functions
%%writefile text_processor/processing.py

#### THIS IMPORT WAS ADDED BY AUTHOR
from text_processor.cleaning import clean_line

def write_words_to_file(words, output_file):
    # Write each word to the output file
    for word in words:
        output_file.write(word + '\n')

def count_word_occurrences(words):
    # Count the occurrences of each word using a dictionary
    word_counts = {}
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

    return word_counts

def print_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n most common words and their occurrences
    print(f"{n} most common words:")
    for word, count in sorted_word_counts[:n]:
        print(f"{word}: {count}")

def print_least_common_words(word_counts, n):
    # Sort the word counts in descending order
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

    # Print the n least common words and their occurrences
    print(f"{n} least common words:")
    for word, count in sorted_word_counts[-n:]:
        print(f"{word}: {count}")

def process_file(input_file_path, output_file_path):
    # Open the input file for reading
    with open(input_file_path, 'r') as input_file:
        # Open the output file for writing
        with open(output_file_path, 'w') as output_file:
            # Iterate over each line in the input file
            for line in input_file:
                # Clean the line
                cleaned_line = clean_line(line)

                # Split the line into words
                words = cleaned_line.split()

                # Write the words to the output file
                write_words_to_file(words, output_file)


Overwriting text_processor/processing.py


In [13]:
# Create an __init__.py file to make the directory a package
%%writefile text_processor/__init__.py
from .cleaning import clean_line
from .processing import process_file, count_word_occurrences, print_common_words, print_least_common_words

Writing text_processor/__init__.py


In [14]:
# Refactor main() to use the new package
%%writefile main.py
import text_processor

input_file_path = 'moby_01.txt'
output_file_path = 'moby_01_clean.txt'

# Process the file
text_processor.process_file(input_file_path, output_file_path)

# Open the file
with open(output_file_path, 'r') as file:
    # Read the file content
    content = file.read()

# Split the content into words
words = content.split()

# Count word occurrences
word_counts = text_processor.count_word_occurrences(words)

# Print the five most common words and their occurrences
text_processor.print_common_words(word_counts, 5)

# Print the five least common words and their occurrences
text_processor.print_least_common_words(word_counts, 5)

Writing main.py


In [3]:
# Run the main script
!python main.py

5 most common words:
the: 14
and: 9
i: 9
of: 8
is: 7
5 least common words:
land: 1
look: 1
at: 1
crowds: 1
watergazers: 1
